In [121]:
import dill as pickle
import os
from pprint import pprint as pp
import json
import pandas as pd
from pandas.api.types import CategoricalDtype
from copy import deepcopy
from tqdm.notebook import tqdm_notebook as progress_bar
from pathlib import Path
import numpy as np
from datetime import datetime, timedelta

In [198]:
main_path = Path('../bt_data_runs/_temp_df.pkl').resolve()
main_df = pd.read_pickle(main_path)
# main_df = type_raw_dataframe(main_df)
# main_df.to_pickle(main_path)


In [149]:
main_df.head()

,buy_date,sell_date,buy_price,sale_price,currency_id,run_name,status,sell_price_multiple,buy_volume_multiple,volume_avg_duration,transparent
0,2018-10-30,2021-04-04,0.381455,1.525818,0XBTC,"0XBTC(10,4,6.0,True)",closed,4,6,10,True
1,2018-10-30,2021-04-05,0.381455,2.288728,0XBTC,"0XBTC(10,6,6.0,True)",closed,6,6,10,True
2,2018-10-30,2021-11-22,0.381455,3.051637,0XBTC,"0XBTC(10,8,6.0,True)",closed,8,6,10,True
3,2018-10-30,1900-01-01,0.381455,-1.000000,0XBTC,"0XBTC(10,10,6.0,True)",opened,10,6,10,True
4,2018-10-30,1900-01-01,0.381455,-1.000000,0XBTC,"0XBTC(10,12,6.0,True)",opened,12,6,10,True


In [199]:
main_df.dtypes

buy_date                       object
sell_date              datetime64[ns]
buy_price                     float64
sale_price                     object
currency_id                    object
run_name                       object
status                         object
sell_price_multiple            object
buy_volume_multiple            object
volume_avg_duration            object
transparent                    object
dtype: object

### Functions:
<ol>
<li>A function to update the datatypes of the different columns.-done</li>
<li>A function to calculate profit of the trade for a given row.-done</li>
<li>A function to calculate present price of the currency if the position was not closed.-done</li>
<li>A function to calculate duration of trade in days.-done</li>
<li>A function to calculate ROI.-done</li>
<li>A function to calculate ROI as % for 30 days.-done</li>
<li>A function to return the price of a currency X days after initiating buy. Error handling for not enough data.-done</li>
</ol>

### Aggregate Analysis
<ol>
<li>Total Profit for all trades for a given set of parameters. New df with parameters and profits.</li>
<li>Avg Profit for all trades with a given set of parameters.</li>
<li>Number of trades executed for a given set of parameters.</li>
<li>A template to filter out trades open longer than X days.</li>
</ol>

In [196]:
def apply_functions(raw_df: pd.DataFrame) -> pd.DataFrame:
    df = type_raw_dataframe(raw_df)
    df['sale_price']=df.apply(fill_in_open_positions, axis=1)
    df['profit'] = df.apply(calculate_profit, axis=1)
    MaxDate = max_date(df)
    df['duration'] = df.apply(calculate_duration, axis=1, max_date=MaxDate).astype('int16')
    # get rid of 0 priced positions
    df = df[df['buy_price'] > 0]
    df['ROI'] = df.apply(calculate_ROI, axis=1).astype('float16')
    df['monthly_ROI'] = df.apply(calculate_monthly_ROI, axis=1).astype('float16')
    return df

def type_raw_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    cat_status = CategoricalDtype(
        categories=['closed', 'opened'], 
        ordered=True,
        )
    return_df = df.astype(
        {
            # 'buy_date': np.datetime64,
            # 'sell_date': np.datetime64,
            'buy_price': 'float64',
            'sale_price': 'float64',
            'currency_id': 'str',
            'run_name': str,
            'status': cat_status,
            'sell_price_multiple': 'int16',
            'buy_volume_multiple': 'int16',
            'volume_avg_duration': 'int16',
            'transparent': bool,

        }, copy=True
    )
    return_df['buy_date'] = pd.to_datetime(return_df['buy_date'], infer_datetime_format=True)
    return_df['sell_date'] = pd.to_datetime(return_df['sell_date'], infer_datetime_format=True)
    return return_df

def calculate_profit(row: pd.DataFrame) -> float:
    if row['sale_price'] != -1:
        return row['sale_price'] - row['buy_price']
    else:
        raise ValueError('Sale price not updated, aborting...')

def fill_in_open_positions(row: pd.DataFrame, dated:int=0) -> float:
    if row['sale_price'] > 0:
        return row['sale_price']
    currency = row['currency_id']
    candle_location = Path(f'../downloaded_data/nomics/candles/{currency}.pkl').resolve()
    currency_df = pd.read_pickle(candle_location)
    if dated == 0:
        # return the last candle close price
        return currency_df['close'].iloc[-1]
    else:
        # return the candle close price X days after the buy date
        check_date = pd.to_datetime(row['buy_date'] + pd.Timedelta(days=int(dated)))
        if check_date > currency_df['timestamp'].iloc[-1]:
            raise ValueError(f'{currency} is not available for {check_date}')
        else:
            return currency_df.loc[currency_df['timestamp'] == check_date]['close'].iloc[0]

def max_date(df = pd.DataFrame) -> datetime:
    currency = df['currency_id'].iloc[0]
    candle_location = Path(f'../downloaded_data/nomics/candles/{currency}.pkl').resolve()
    currency_df = pd.read_pickle(candle_location)
    return currency_df['timestamp'].iloc[-1]

def calculate_duration(df: pd.DataFrame, max_date: datetime) -> int:
    if df['status'] == 'opened':
        return (max_date - df['buy_date']).days
    return (df['sell_date'] - df['buy_date']).days

def calculate_ROI(df: pd.DataFrame) -> float:
    net_return = df['profit']-df['buy_price']
    return net_return/df['buy_price']*100

def calculate_monthly_ROI(df: pd.DataFrame) -> float:
    # only run after calculating ROI
    months_held = df['duration']/30
    monthly_ROI = ((df['ROI']+1) ** (1/months_held) - 1) * 100
    return monthly_ROI



In [ ]:
typed_df = apply_functions(main_df)

In [117]:
typed_df.dtypes

buy_date               datetime64[ns]
sell_date              datetime64[ns]
buy_price                     float64
sale_price                    float64
currency_id                    object
run_name                       object
status                       category
sell_price_multiple             int16
buy_volume_multiple             int16
volume_avg_duration             int16
transparent                      bool
profit                        float64
dtype: object

In [103]:
import sys
print("typed",sys.getsizeof(typed_df)/(1024*1024))
print("untyped",sys.getsizeof(main_df)/(1024*1024))

typed 0.0010442733764648438
untyped 345.4046058654785


In [200]:
df = type_raw_dataframe(main_df)

# def test(df: pd.DataFrame) -> pd.DataFrame:
#     currency = df['currency_id']
#     candle_location = Path(f'../downloaded_data/nomics/candles/{currency}.pkl').resolve()
#     currency_df = pd.read_pickle(candle_location)
#     check_date = pd.to_datetime(df['buy_date'] + pd.Timedelta(days=int(30)))
#     print(currency_df.loc[currency_df['timestamp'] == check_date]['close'].iloc[0])

# df['sale_price'] = df.apply(fill_in_open_positions, axis=1)
# MaxDate = max_date(df)
# df['duration'] = df.apply(calculate_duration, axis=1, max_date=MaxDate).astype('int16')
# df = df.astype({'duration': 'int16'}, copy=True)
df.dtypes

buy_date               datetime64[ns]
sell_date              datetime64[ns]
buy_price                     float64
sale_price                    float64
currency_id                    object
run_name                       object
status                       category
sell_price_multiple             int16
buy_volume_multiple             int16
volume_avg_duration             int16
transparent                      bool
dtype: object

In [139]:
df.dtypes

buy_date               datetime64[ns]
sell_date              datetime64[ns]
buy_price                     float64
sale_price                    float64
currency_id                    object
run_name                       object
status                       category
sell_price_multiple             int16
buy_volume_multiple             int16
volume_avg_duration             int16
transparent                      bool
duration                        int16
dtype: object

In [157]:
buy_prices = main_df[main_df['buy_price'] > 0]

In [160]:
buy_prices['buy_price'].describe()

count    2.096364e+06
mean     2.451705e+02
std      2.869816e+03
min      8.800000e-17
25%      1.687128e-03
50%      5.776551e-02
75%      9.382706e-01
max      1.653324e+05
Name: buy_price, dtype: float64

In [ ]:
save_path = Path('../bt_data_runs/archive_df_test.pkl').resolve()
df.to_pickle(save_path)
